<a href="https://colab.research.google.com/github/Alpha-mon/AI-RoboAdvisor/blob/main/Sentiment_Analysis_for_Stock_Market_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 뉴스 제목, 일자 크롤링

import requests
from bs4 import BeautifulSoup
import datetime
import time
import pandas as pd

base_url = "https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=101&date="

# 1. User-Agent를 설정
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# 시작 날짜와 종료 날짜 설정
start_date = datetime.datetime.now() - datetime.timedelta(days=30)
end_date = datetime.datetime.now() - datetime.timedelta(days=5)

current_date = start_date

news_data = []

while current_date <= end_date:
    # 2. 요청을 보낼 때 headers 추가
    response = requests.get(base_url + current_date.strftime('%Y%m%d'), headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    for item in soup.select(".cluster_text a"):
        title = item.text.strip()
        if item.attrs["href"].startswith("http"):
            news_url = item.attrs["href"]
        else:
            news_url = "https:" + item.attrs["href"]

        detail_response = requests.get(news_url, headers=headers)  # headers 추가
        detail_soup = BeautifulSoup(detail_response.text, 'html.parser')
        date_element = detail_soup.select_one("span.media_end_head_info_datestamp_time")
        date = date_element.attrs["data-date-time"].split()[0]

        news_data.append({
            'title': title,
            'date': date
        })

        # 요청 간에 약간의 지연을 두어 IP 차단을 피하기
        time.sleep(1.5)

    # 다음 날짜로 이동
    current_date += datetime.timedelta(days=1)

# 뉴스 데이터를 날짜 순으로 정렬
news_data_sorted = sorted(news_data, key=lambda x: x['date'])


# 데이터프레임으로 변환
news_df = pd.DataFrame(news_data_sorted, columns=['date', 'title'])

# 시작 날짜와 종료 날짜를 문자열로 변환
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

# 원하는 날짜 범위만 선택
news_df = news_df[(news_df['date'] >= start_date_str) & (news_df['date'] <= end_date_str)]

In [3]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 44.4 MB/s eta 0:00:00


In [4]:
# 크롤링한 뉴스 제목 명사 추출

from konlpy.tag import Okt

# Okt 객체 초기화
okt = Okt()

# 제목에서 명사만 추출하는 함수
def extract_nouns(title):
    return ', '.join(okt.nouns(title))

# 'title' 열의 각 제목에 대하여 명사만 추출
news_df['nouns'] = news_df['title'].apply(extract_nouns)

print(news_df)

          date                                          title  \
5   2023-09-19                          삼성전자, 3주만에 다시 '6만전자'로   
6   2023-09-19                          삼성전자, 3주만에 다시 '6만전자'로   
7   2023-09-19                       수요와 균형 이루는 주택공급 확대 정책 기대   
8   2023-09-20                  주택구입자 3분의1이 받은 특례보금자리론…축소 파장은   
9   2023-09-27                    도생·오피스텔에 기금 지원…공급부족 해소 도움될까   
10  2023-09-27                    사라 버튼 떠나는 알렉산더 맥퀸, 후임 누가 될까   
11  2023-09-27                             이더리움 NFT가 지갑이 된다고?   
12  2023-10-03  ‘월클 美모’ 송혜교‧수지, 300만원 vs 1000만원 ‘여신 드레스룩’ 승자는   
13  2023-10-03                    [데스크칼럼]반쪽 주택대책 안되려면 ‘실행’뿐이다   
14  2023-10-04                        디폴트옵션 완전정복…당신도 연금부자 됩니다   
15  2023-10-05                      IPO 대어 두산로보틱스, 상장 첫날 ‘따블’   
16  2023-10-05               "에코프로 욕하려면 80만원 내세요"…확 달라진 '종토방'   
17  2023-10-05                      IPO 대어 두산로보틱스, 상장 첫날 ‘따블’   
18  2023-10-06                    80~1000만원대까지…시선강탈 레드카펫 ★드레스   
19  2023-10-06           

In [5]:
# 코스피 등락율 - 뉴스

import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_kospi_closing_prices():
    url = "https://finance.naver.com/sise/sise_index_day.nhn?code=KOSPI"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    kospi_closings = []

    for i in range(1, 7):
        response = requests.get(url + f"&page={i}", headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        dates = soup.select(".date")
        closings = soup.select(".number_1")

        for d, c in zip(dates, closings[::4]):  # 종가만 가져오기 위해 slicing 사용
            kospi_closings.append([d.text.strip(), float(c.text.replace(',', ''))])

    return kospi_closings

kospi_data = get_kospi_closing_prices()
df = pd.DataFrame(kospi_data, columns=["Date", "Closing"])

# Shift를 사용해 다음 날짜의 종가를 가져와서 현재 날짜와 비교
# 예: 10월 9일 종가보다 10월 10일 종가가 더 높다면 10월 9일 등락율 1이 됨

df["Up/Down"] = (df["Closing"].shift(1) > df["Closing"]).astype(int)

# 날짜 기준으로 최근 30일의 데이터를 가져온 후, 정렬
df = df.sort_values(by="Date").tail(30).reset_index(drop=True)

df['Date'] = pd.to_datetime(df['Date'], format='%Y.%m.%d').dt.strftime('%Y-%m-%d')

print(df)

merged_df = pd.merge(news_df, df, left_on='date', right_on='Date', how='inner')
merged_df = merged_df[['date', 'Up/Down', 'title', 'nouns']]
print(merged_df)

          Date  Closing  Up/Down
0   2023-08-31  2556.27        1
1   2023-09-01  2563.71        1
2   2023-09-04  2584.55        0
3   2023-09-05  2582.18        0
4   2023-09-06  2563.34        0
5   2023-09-07  2548.26        0
6   2023-09-08  2547.68        1
7   2023-09-11  2556.88        0
8   2023-09-12  2536.58        0
9   2023-09-13  2534.70        1
10  2023-09-14  2572.89        1
11  2023-09-15  2601.28        0
12  2023-09-18  2574.72        0
13  2023-09-19  2559.21        1
14  2023-09-20  2559.74        0
15  2023-09-21  2514.97        0
16  2023-09-22  2508.13        0
17  2023-09-25  2495.76        0
18  2023-09-26  2462.97        1
19  2023-09-27  2465.07        0
20  2023-10-04  2405.69        0
21  2023-10-05  2403.60        1
22  2023-10-06  2408.73        0
23  2023-10-10  2402.58        1
24  2023-10-11  2450.08        1
25  2023-10-12  2479.82        0
26  2023-10-13  2456.15        0
27  2023-10-16  2436.24        1
28  2023-10-17  2460.17        1
29  2023-1

In [8]:
# 단어 점수 초기화 및 개수
# 한 글자 제외 -> 두 글자 이상 단어

# 'nouns' 칼럼의 값을 문자열로 변환
df['nouns'] = merged_df['nouns'].astype(str)
df = df.dropna(subset=['nouns'])

# 'filtered_nouns' 컬럼 생성
merged_df['filtered_nouns'] = merged_df['nouns'].apply(lambda x: [word for word in x.split(', ') if len(word) > 1])

# 단어 점수 초기화
word_scores = {word: 0 for word_list in merged_df['filtered_nouns'] for word in word_list}
print(word_scores)

# 단어 빈도수 계산
from collections import Counter
word_counts = Counter(word for word_list in merged_df['filtered_nouns'] for word in word_list)

# 결과 출력
for word, count in word_counts.items():
    print(f"{word}: {count}")

{'삼성': 0, '전자': 0, '다시': 0, '수요': 0, '균형': 0, '주택': 0, '공급': 0, '확대': 0, '정책': 0, '기대': 0, '구입': 0, '특례': 0, '보금자리': 0, '축소': 0, '파장': 0, '도생': 0, '오피스텔': 0, '기금': 0, '지원': 0, '부족': 0, '해소': 0, '도움': 0, '사라': 0, '버튼': 0, '알렉산더': 0, '맥퀸': 0, '후임': 0, '누가': 0, '리움': 0, '지갑': 0, '디폴트': 0, '옵션': 0, '완전': 0, '정복': 0, '당신': 0, '연금': 0, '부자': 0, '두산': 0, '로보틱스': 0, '상장': 0, '첫날': 0, '따블': 0, '에코': 0, '프로': 0, '내세': 0, '토방': 0, '시선': 0, '강탈': 0, '레드카펫': 0, '드레스': 0, '모두': 0, '백인': 0, '남성': 0, '명품': 0, '기업': 0, '케링': 0, '그룹': 0, '이유': 0, '빗썸': 0, '수수료': 0, '무료': 0, '프로젝트': 0, '덕분': 0, '단위': 0, '영업': 0, '복귀': 0, '강세': 0, '데스크': 0, '칼럼': 0, '먹거리': 0, '물가': 0, '기후': 0, '위기': 0, '욕구': 0, '폭발': 0, '뉴진스': 0, '팝업': 0, '스토어': 0, '대박': 0, '목욕탕': 0, '슬리퍼': 0, '회사': 0, '버켄스탁': 0, '당첨': 0, '중복': 0, '청약': 0, '거주': 0, '의무': 0, '마곡': 0, '하남': 0, '뉴홈': 0, '가구': 0, '매매': 0, '전세': 0, '이제': 0, '동행': 0, '고조': 0, '전쟁': 0, '우려': 0, '코인': 0, '정말': 0, '계획': 0, '최태원': 0, '구상': 0, '후계': 0, '구도': 0, '판매': 0, '쏘렌토': 0, '파

In [11]:
# 단어 점수 부여

# 전체 단어 개수 출력
total_words = sum(word_counts.values())
print(f"\nTotal number of words: {total_words}")

# Up/Down 값이 1인 데이터에서 포함된 단어의 리스트
up = []
for nouns in merged_df[merged_df['Up/Down'] == 1]['filtered_nouns']:
    up.extend(nouns)

# Up/Down 값이 0인 데이터에서 포함된 단어의 리스트
down = []
for nouns in merged_df[merged_df['Up/Down'] == 0]['filtered_nouns']:
    down.extend(nouns)

print("up :", len(up))
print("down :", len(down))

# 상승 비율과 하락 비율 계산
total_words = len(up) + len(down)
up_ratio = len(up) / total_words
down_ratio = len(down) / total_words

# 단어 점수 초기화
word_scores = {word: 0 for word in word_scores.keys()}  # 기존의 word_scores 딕셔너리 사용

# Up(1) 데이터의 단어들에 대해서 하락 비율을 더해주기
for word in up:
    if word in word_scores:
        word_scores[word] += down_ratio

# Down(0) 데이터의 단어들에 대해서 상승 비율을 차감해주기
for word in down:
    if word in word_scores:
        word_scores[word] -= up_ratio

# 결과 확인
print(word_scores)


Total number of words: 198
up : 75
down : 123
{'삼성': 2.484848484848485, '전자': 2.484848484848485, '다시': 1.2424242424242424, '수요': 0.6212121212121212, '균형': 0.6212121212121212, '주택': -0.13636363636363635, '공급': 0.24242424242424243, '확대': 0.6212121212121212, '정책': 0.6212121212121212, '기대': 0.6212121212121212, '구입': -0.3787878787878788, '특례': -0.3787878787878788, '보금자리': -0.3787878787878788, '축소': -0.3787878787878788, '파장': -0.3787878787878788, '도생': -0.3787878787878788, '오피스텔': -0.3787878787878788, '기금': -0.3787878787878788, '지원': -0.3787878787878788, '부족': -0.3787878787878788, '해소': -0.3787878787878788, '도움': -0.3787878787878788, '사라': -0.3787878787878788, '버튼': -0.3787878787878788, '알렉산더': -0.3787878787878788, '맥퀸': -0.3787878787878788, '후임': -0.3787878787878788, '누가': -0.3787878787878788, '리움': -0.3787878787878788, '지갑': -0.3787878787878788, '디폴트': -0.3787878787878788, '옵션': -0.3787878787878788, '완전': -0.3787878787878788, '정복': -0.3787878787878788, '당신': -0.3787878787878788, '연금': -0.

In [12]:
# 감성 사전 완료

total = []
for nouns in merged_df['filtered_nouns']:
    sent_score = 0
    for noun in nouns:
        if noun in word_scores:
            sent_score += word_scores[noun]

    # 해당 뉴스 제목에 포함된 단어의 수로 나누어 평균 점수를 계산
    avg_sent_score = sent_score / len(nouns) if nouns else 0  # 단어가 없는 경우 0으로 처리
    total.append(avg_sent_score)

merged_df['sent_score'] = total

# 감성사전의 평균 점수 계산

sent_mean = sum(word_scores.values()) / len(word_scores)
print('감성 사전 평균 점수 : ',sent_mean)

# 감성 점수 계산
def calculate_sentiment_score(noun_list):
    score = 0
    for noun in noun_list:
        if noun in word_scores:
            score += word_scores[noun]
    return score / (len(noun_list) if len(noun_list) != 0 else 1)

merged_df['sent_score'] = merged_df['filtered_nouns'].apply(calculate_sentiment_score)

# 평균 점수를 기준으로 라벨링
merged_df['sent_label'] = merged_df['sent_score'].apply(lambda x: 1 if x > sent_mean else 0)



result_df = merged_df[['date', 'Up/Down', 'sent_score', 'sent_label', 'title', 'nouns']]
print(result_df)

감성 사전 평균 점수 :  -8.61974398000898e-17
          date  Up/Down  sent_score  sent_label  \
0   2023-09-19        1    2.070707           1   
1   2023-09-19        1    2.070707           1   
2   2023-09-19        1    0.458874           1   
3   2023-09-20        0   -0.338384           0   
4   2023-09-27        0   -0.301136           0   
5   2023-09-27        0   -0.378788           0   
6   2023-09-27        0   -0.378788           0   
7   2023-10-04        0   -0.378788           0   
8   2023-10-05        1    1.242424           1   
9   2023-10-05        1    0.621212           1   
10  2023-10-05        1    1.242424           1   
11  2023-10-06        0   -0.378788           0   
12  2023-10-06        0   -0.443182           0   
13  2023-10-06        0   -0.378788           0   
14  2023-10-11        1    1.530303           1   
15  2023-10-11        1    1.530303           1   
16  2023-10-11        1    0.724747           1   
17  2023-10-11        1    0.512987          

In [15]:
# 모델링

from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# 1. 데이터 준비
X_data = merged_df['nouns'].apply(lambda x: ' '.join([word for word in x.split(', ') if len(word) > 1])).values
Y_data = merged_df['sent_label'].values  # 기존의 'merged_df'를 사용

# 2. 토큰화 및 패딩
vocab_size = 2000
tokenizer = Tokenizer(num_words=vocab_size, oov_token='OOV')
tokenizer.fit_on_texts(X_data)
X_tokenized = tokenizer.texts_to_sequences(X_data)
X_padded = pad_sequences(X_tokenized, maxlen=30)

# 3. Bi-LSTM 모델 구축 및 훈련
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

history = model.fit(X_padded, Y_data, epochs=15, callbacks=[es, mc], batch_size=256, validation_split=0.2)

Epoch 1/15
1/1 [==============================] - ETA: 0s - loss: 0.6937 - acc: 0.4194
Epoch 1: val_acc improved from -inf to 1.00000, saving model to best_model.h5
1/1 [==============================] - 9s 9s/step - loss: 0.6937 - acc: 0.4194 - val_loss: 0.6604 - val_acc: 1.0000
Epoch 2/15
1/1 [==============================] - ETA: 0s - loss: 0.6911 - acc: 0.5161
Epoch 2: val_acc did not improve from 1.00000
1/1 [==============================] - 0s 83ms/step - loss: 0.6911 - acc: 0.5161 - val_loss: 0.6604 - val_acc: 1.0000
Epoch 3/15
1/1 [==============================] - ETA: 0s - loss: 0.6893 - acc: 0.5161
Epoch 3: val_acc did not improve from 1.00000
1/1 [==============================] - 0s 93ms/step - loss: 0.6893 - acc: 0.5161 - val_loss: 0.6599 - val_acc: 1.0000
Epoch 4/15
1/1 [==============================] - ETA: 0s - loss: 0.6874 - acc: 0.5161
Epoch 4: val_acc did not improve from 1.00000
1/1 [==============================] - 0s 99ms/step - loss: 0.6874 - acc: 0.5161 - v

In [17]:
# 최신 뉴스 크롤링 후 모델에 적용해서 최종 결과 확

# 뉴스 제목 크롤링
base_url = "https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=101&date="
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

start_date = datetime.datetime.now() - datetime.timedelta(days=30)
end_date = datetime.datetime.now() - datetime.timedelta(days=5)
current_date = start_date

news_data = []

while current_date <= end_date:
    response = requests.get(base_url + current_date.strftime('%Y%m%d'), headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    for item in soup.select(".cluster_text a"):
        title = item.text.strip()
        news_data.append({
            'date': current_date.strftime('%Y-%m-%d'),
            'title': title
        })
    current_date += datetime.timedelta(days=1)

news_df = pd.DataFrame(news_data, columns=['date', 'title'])

# 명사 추출
okt = Okt()
news_df['nouns'] = news_df['title'].apply(lambda x: ', '.join(okt.nouns(x)))

# 감성 사전과 비교하여 감성 점수 계산
news_df['filtered_nouns'] = news_df['nouns'].apply(lambda x: [word for word in x.split(', ') if len(word) > 1])
news_df['sent_score'] = news_df['filtered_nouns'].apply(calculate_sentiment_score)  # 이전에 정의한 함수

# 예측을 위한 데이터 전처리
X_test_tokenized = tokenizer.texts_to_sequences(news_df['nouns'].apply(lambda x: ' '.join([word for word in x.split(', ') if len(word) > 1])).values)
X_test_padded = pad_sequences(X_test_tokenized, maxlen=30)

# 훈련된 Bi-LSTM 모델로 예측
predicted = model.predict(X_test_padded)
news_df['predicted_label'] = (predicted > 0.5).astype(int)

# 결과 출력
positive_news_ratio = news_df['predicted_label'].sum() / len(news_df)
if positive_news_ratio > 0.5:
    print("미래 주식 시장은 긍정적으로 움직일 것으로 예상됩니다.")
else:
    print("미래 주식 시장은 부정적으로 움직일 것으로 예상됩니다.")

    # 각 뉴스의 감성 점수를 바탕으로 긍정적 및 부정적 뉴스 개수 확인 및 출력
sent_mean = news_df['sent_score'].mean()

pos_news = len(news_df[news_df['sent_score'] > sent_mean])
neg_news = len(news_df[news_df['sent_score'] <= sent_mean])
total_news = len(news_df)

print(f"긍정적 뉴스 수: {pos_news} ({pos_news/total_news*100:.2f}%)")
print(f"부정적 뉴스 수: {neg_news} ({neg_news/total_news*100:.2f}%)")

print("\n긍정적 뉴스 예시:")
for title in news_df[news_df['sent_score'] > sent_mean]['title'].head(5):
    print("-", title)

print("\n부정적 뉴스 예시:")
for title in news_df[news_df['sent_score'] <= sent_mean]['title'].head(5):
    print("-", title)

4/4 [==============================] - 0s 19ms/step
미래 주식 시장은 부정적으로 움직일 것으로 예상됩니다.
긍정적 뉴스 수: 21 (20.19%)
부정적 뉴스 수: 83 (79.81%)

긍정적 뉴스 예시:
- 뉴:홈, 당첨일 다르면 중복청약 OK…거주 의무도 꼼꼼히
- 마곡 3억·하남 4억대 ‘뉴홈’ 3300가구 풀린다
- 청약 경쟁률 수십대 1인데…왜 '완판'은 안될까?
- 삼성전자,한 달 만에 ‘7만전자’복귀
- 삼성전자, 3분기 ‘조 단위’ 영업익 복귀에 강세

부정적 뉴스 예시:
- 만기가 돌아왔다…갈아타기 고민하는 예테크족
- 디폴트옵션 완전정복…당신도 연금부자 됩니다
- "차량 3대면 주차비 20만원 내세요" 주차난에 차등 요금 도입 단지↑
- 치솟는 강남권 초소형 아파트 인기… 일부 단지 신고가 근접
- “MZ는 주택시장 풍향계?”…서울집 내다 파는 2030 늘고, 영끌족 줄어
